In [1]:
import pandas as pd
import numpy as np
import re
import datetime as dt
from pivottablejs import pivot_ui
import pivottablejs
import qgrid
import matplotlib.pyplot as plt

In [2]:
path='C:\\Users\\camendol\\Desktop\\'
file='PCRs.xlsx'

file_path=path+file

In [3]:
#Open connection to EXCEL
xl_file = pd.ExcelFile(file_path)
xl_file.sheet_names 

['PCRs']

In [4]:
#Read a tab
pcr_data=xl_file.parse('PCRs')

In [5]:
#Apply Date typing
pcr_data['Created']=pd.to_datetime(pcr_data['Creation Date'].str[0:20])
pcr_data['Created_yr_mon'] = pd.to_datetime(pcr_data['Created'].dt.date).map(lambda dt: dt.replace(day=1))
pcr_data['Created_yr'] = pd.to_datetime(pcr_data['Created'].dt.date).map(lambda dt: dt.replace(day=1,month=1))
pcr_data['Updated']=pd.to_datetime(pcr_data['Last Update Date'].str[0:20])
pcr_data['Duration']=pcr_data['Updated'] - pcr_data['Created']
pcr_data['Duration']=pcr_data['Duration']/np.timedelta64(1,'D')

pcr_data['QA Expected']=pd.to_datetime(pcr_data['Expected Date to QA'].str[0:20])
pcr_data['QA Delivered']=pd.to_datetime(pcr_data['Actual Date to QA'].str[0:20])
pcr_data['PCR Approved']=pd.to_datetime(pcr_data['New PCR Approval Date'].str[0:20]) 

pcr_data['Update_Elapsed']=(dt.datetime.today() - pcr_data['Updated']).dt.days 
#Maybe bin off day buckets
bin_30s = [-1,15,30,45, 60, 75 ,90, 120, 150, 180, 210, 240, 270, 300, 330, 360, np.inf]
bin_labels = ['15','30', '45','60','75' ,'90', '120', '150','180','210','240','270','300','330','360','361+']
pcr_data['Updated_Seg']=pd.cut(pcr_data['Update_Elapsed'],bin_30s,labels=bin_labels)
pcr_data['Duration_Seg']=pd.cut(pcr_data['Duration'],bin_30s,labels=bin_labels)
#pcr_data.loc[pcr_data['Updated_Seg'].isna()]
#Week Bins
week_bin = [-1,7,14,21,28,35,42,49,56,63,70,77,84,91,98,105,112,119,126,133,140,147,154,161,168,175,182,np.inf]
week_labels = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27+']
pcr_data['Updated_WKs']=pd.cut(pcr_data['Update_Elapsed'],week_bin,labels=week_labels)

In [6]:
current_team=[ 'Lois Nordstrom'\
              , 'Sue Szubzda'\
              ,'Tricia Huebner'\
              , 'Padmaja Gandhi'\
              ,'Phanikumar Jetti'\
              , 'Pragada Kumar'\
              , 'Christopher Amendola'\
              ,''\
              ,' ']

In [7]:
#Created:Year-to-date
created_YTD=pcr_data.loc[ ((pcr_data['Owner'].isin(current_team) | pcr_data['Owner'].isnull())
                               & (pcr_data['Created']>'2020-01-01'))\
                               ,( 'Formatted ID'\
                                 ,'Created'\
                                 ,'Updated'\
                                 ,'Created_yr_mon'\
                                 ,'Created_yr'\
                                 ,'Updated_Seg'\
                                 ,'Updated_WKs'\
                                 ,'Duration_Seg'\
                                 ,'Duration'\
                                 ,'Name'\
                                 ,'Impact Kanban State'\
                                 ,'Owner')]\
        .sort_values(by='Created')
len(created_YTD)

128

In [8]:
#Created:Year-to-date
created_hist=pcr_data.loc[ ((pcr_data['Owner'].isin(current_team) | pcr_data['Owner'].isnull())
                               & (pcr_data['Created']>'2019-01-01'))\
                               ,( 'Formatted ID'\
                                 ,'Created'\
                                 ,'Updated'\
                                 ,'Created_yr_mon'\
                                 ,'Created_yr'\
                                 ,'Updated_Seg'\
                                 ,'Updated_WKs'\
                                 ,'Duration_Seg'\
                                 ,'Duration'\
                                 ,'Name'\
                                 ,'Impact Kanban State'\
                                 ,'Owner')]\
        .sort_values(by='Created')
len(created_hist)

251

In [9]:
completed_20=pcr_data.loc[( (pcr_data['Owner'].isin(current_team))\
                             &  (pcr_data['Impact Kanban State']=='Released')\
                             | (pcr_data['Impact Kanban State']=='UAT'))\
                             & (pcr_data['Created_yr']=='2020-01-01')                            
                          ,( 'Formatted ID'\
                                 ,'Created'\
                                 ,'Updated'\
                                 ,'Created_yr_mon'\
                                 ,'Duration_Seg'\
                                 ,'Duration'\
                                 ,'Name'\
                                 ,'Impact Kanban State'\
                                 ,'Owner')]
len(completed_20)

90

In [10]:
active=pcr_data.loc[( (pcr_data['Owner'].isin(current_team))\
                          &((pcr_data['Impact Kanban State']=='Dev') \
                          | (pcr_data['Impact Kanban State']=='Approval')\
                          | (pcr_data['Impact Kanban State']=='Requests')
                          | (pcr_data['Impact Kanban State']=='Code Complete')
                          | (pcr_data['Impact Kanban State']=='Pending Approval')))\
                          ,( 'Formatted ID'\
                                 ,'Created'\
                                 ,'Updated'\
                                 ,'Created_yr_mon'\
                                 ,'Created_yr'\
                                 ,'Updated_Seg'\
                                 ,'Updated_WKs'\
                                 ,'Duration_Seg'\
                                 ,'Duration'\
                                 ,'Name'\
                                 ,'Impact Kanban State'\
                                 ,'Owner')]

In [11]:
print('Active Requests 2020: '+str(len(active)))

Active Requests 2020: 44


### Active Requests 2020: 
###                                        37 - 06.02.20
###                                        34 - 07.27.20
###                                        33 - 07.30.20
###                                        37 - 08.20.20
###                                        35 - 09.11.20
###                                        44 - 10.15.20

In [12]:
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [15, 5]

In [13]:
pivot_ui(completed_20,rows=['Duration_Seg'],rendererName='Table')

In [14]:
pivot_ui(active,rows=['Updated_WKs'],rendererName='Table')

In [15]:
pivot_ui(created_hist,rows=['Created_yr'],cols=['Created_yr_mon'],rendererName='Bar Chart')

In [16]:
# qgrid.show_grid(active[[ 'Name'\
#                           ,'Formatted ID'\
#                           ,'Owner'\
#                           ,'Updated_WKs'\
#                           ,'Created_yr'\
#                           ,'Duration'\
#                           ,'Impact Kanban State'\
#                           ]],show_toolbar=True)

In [17]:
#Create Excel sheets for each person
for whom in active['Owner'].unique():
    
    excel_book='C:\\Users\\camendol\\Documents\\PCRs\\'+whom+'.xlsx'
    current=active.loc[(active['Owner']==whom)\
                       ,(  'Name'\
                          ,'Formatted ID'\
                          ,'Owner'\
                          ,'Updated_WKs'\
                          ,'Created_yr'\
                          ,'Duration'\
                          ,'Impact Kanban State')]
    
    comps=completed_20.loc[(completed_20['Owner']==whom),('Name','Formatted ID','Owner','Created_yr_mon','Duration','Impact Kanban State')]

    print(excel_book)
    with pd.ExcelWriter(excel_book) as writer:
        pd.DataFrame(current)\
        .to_excel(writer, sheet_name='Active')
        pd.DataFrame(comps)\
        .to_excel(writer, sheet_name='Completed')

C:\Users\camendol\Documents\PCRs\Sue Szubzda.xlsx
C:\Users\camendol\Documents\PCRs\Lois Nordstrom.xlsx
C:\Users\camendol\Documents\PCRs\Christopher Amendola.xlsx
C:\Users\camendol\Documents\PCRs\Tricia Huebner.xlsx
C:\Users\camendol\Documents\PCRs\Padmaja Gandhi.xlsx
C:\Users\camendol\Documents\PCRs\Pragada Kumar.xlsx
